# Attempt to fine-tune GPT3 to translate between languages
The first language will be our made up language (see notes on get_bible as to why we think GPT is cheating)
The thesis is that fine-tuning will cause GPT3 to learn the fake language as a new language by assigning the embeddings of the
new words essentially to the same dimensional space as the english.  Should be trivial to translate it back as it is a word 
for word translation.



In [1]:
from lib.config import get_config
import pandas as pd

bible = pd.read_csv('data/bible.bbe.csv')

# Assuming the gospels have a lot of overlap include the synoptic gospels plus John to train
# then predict Mark (which should be easy as Matthew and Luke may have copied from him)
train = bible[bible['book'].isin(['EXO'])]
validate = bible[bible['book'].isin(['MRK'])]

In [2]:
train

,0,book,chapter,verse,eng-web,eng-asv,eng-kjv2006,engBBE,engWycliffe,birrig
1533,EXO 1:1,EXO,1,1,"Now these are the names of the sons of Israel,...","Now these are the names of the sons of Israel,...",Now these are the names of the children of Isr...,Now these are the names of the sons of Israel ...,"These ben the names of the sones of Israel, th...",Ruh lxizi evi lxi reniz us lxi zurz us Ozveip ...
1534,EXO 1:2,EXO,1,2,"Reuben, Simeon, Levi, and Judah,","Reuben, Simeon, Levi, and Judah,","Reuben, Simeon, Levi, and Judah,","Reuben, Simeon, Levi, and Judah;","Ruben, Symeon,","Viafir, Zoniur, Pimo, erw Yawex;"
1535,EXO 1:3,EXO,1,3,"Issachar, Zebulun, and Benjamin,","Issachar, Zebulun, and Benjamin,","Issachar, Zebulun, and Benjamin,","Issachar, Zebulun, and Benjamin;","Leuy, Judas, Isachar, Zabulon, and Benjamin,","Ozzekxev, Difapar, erw Firyenor;"
1536,EXO 1:4,EXO,1,4,"Dan and Naphtali, Gad and Asher.","Dan and Naphtali, Gad and Asher.","Dan, and Naphtali, Gad, and Asher.","Dan and Naphtali, Gad and Asher.","Dan, and Neptalim, Gad, and Aser.","Wer erw Reqxlepo, Gew erw Ezxiv."
1537,EXO 1:5,EXO,1,5,All the souls who came out of Jacob’s body wer...,And all the souls that came out of the loins o...,And all the souls that came out of the loins o...,All the offspring of Jacob were seventy person...,Therfor alle the soules of hem that yeden out ...,Epp lxi usszqvorg us Yekuf hivi zimirlj qivzur...
...,...,...,...,...,...,...,...,...,...,...
2741,EXO 40:34,EXO,40,34,"Then the cloud covered the Tent of Meeting, an...","Then the cloud covered the tent of meeting, an...",¶ Then a cloud covered the tent of the congreg...,Then the cloud came down covering the Tent of ...,"a cloude hilide the tabernacle of witnessyng, ...",Lxir lxi kpuaw keni wuhr kumivorg lxi Lirl us ...
2742,EXO 40:35,EXO,40,35,Moses wasn’t able to enter into the Tent of Me...,And Moses was not able to enter into the tent ...,And Moses was not able to enter into the tent ...,So that Moses was not able to go into the Tent...,nether Moyses myyte entre in to the tabernacle...,Zu lxel Nuziz hez rul efpi lu gu orlu lxi Lirl...
2743,EXO 40:36,EXO,40,36,When the cloud was taken up from over the tabe...,And when the cloud was taken up from over the ...,And when the cloud was taken up from over the ...,And whenever the cloud was taken up from the H...,"If ony tyme the cloude lefte the tabernacle, t...",Erw hxirimiv lxi kpuaw hez lecir aq svun lxi X...
2744,EXO 40:37,EXO,40,37,"but if the cloud wasn’t taken up, then they di...","but if the cloud was not taken up, then they j...","But if the cloud were not taken up, then they ...","But while the cloud was there, they made no mo...","if the cloude hangide aboue, thei dwelliden in...","Fal hxopi lxi kpuaw hez lxivi, lxij newi ru nu..."


In [3]:
df2 = pd.DataFrame()
for item in ['eng-web', 'eng-asv', 'eng-kjv2006']:
    df2 = df2.append(pd.DataFrame({'prompt': train[item], 'completion': train['birrig']}))

# removal all null rows
df2 = df2.dropna()

df2['completion'] = df2['completion'].apply(lambda x: " " + x + '\n###')
df2['prompt'] = df2['prompt'].apply(lambda x: x + "\n\n###\n\n")
df2.to_json('data/train_exp_exo.jsonl', orient='records', lines=True)
df2


/var/folders/tq/94d0g6xx05x_h_z4rfdfh6bh0000gn/T/ipykernel_69408/1281868063.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame({'prompt': train[item], 'completion': train['birrig']}))
/var/folders/tq/94d0g6xx05x_h_z4rfdfh6bh0000gn/T/ipykernel_69408/1281868063.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame({'prompt': train[item], 'completion': train['birrig']}))
/var/folders/tq/94d0g6xx05x_h_z4rfdfh6bh0000gn/T/ipykernel_69408/1281868063.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame({'prompt': train[item], 'completion': train['birrig']}))


,prompt,completion
1533,"Now these are the names of the sons of Israel,...",Ruh lxizi evi lxi reniz us lxi zurz us Ozveip...
1534,"Reuben, Simeon, Levi, and Judah,\n\n###\n\n","Viafir, Zoniur, Pimo, erw Yawex;\n###"
1535,"Issachar, Zebulun, and Benjamin,\n\n###\n\n","Ozzekxev, Difapar, erw Firyenor;\n###"
1536,"Dan and Naphtali, Gad and Asher.\n\n###\n\n","Wer erw Reqxlepo, Gew erw Ezxiv.\n###"
1537,All the souls who came out of Jacob’s body wer...,Epp lxi usszqvorg us Yekuf hivi zimirlj qivzu...
...,...,...
2741,¶ Then a cloud covered the tent of the congreg...,Lxir lxi kpuaw keni wuhr kumivorg lxi Lirl us...
2742,And Moses was not able to enter into the tent ...,Zu lxel Nuziz hez rul efpi lu gu orlu lxi Lir...
2743,And when the cloud was taken up from over the ...,Erw hxirimiv lxi kpuaw hez lecir aq svun lxi ...
2744,"But if the cloud were not taken up, then they ...","Fal hxopi lxi kpuaw hez lxivi, lxij newi ru n..."


In [4]:

# set environment variable in ipython notebook
import os
os.environ["OPENAI_API_KEY"] = get_config('openai')['api_key']




In [5]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py tools fine_tunes.prepare_data -f data/train_exp_exo.jsonl -q

Analyzing...

- Your file contains 3639 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 156 duplicated prompt-completion sets. These are rows: [335, 357, 425, 663, 860, 866, 888, 899, 1175, 1213, 1214, 1215, 1216, 1324, 1352, 1543, 1548, 1570, 1638, 1657, 1781, 1876, 1936, 1966, 1992, 2073, 2079, 2101, 2112, 2278, 2328, 2356, 2388, 2427, 2428, 2431, 2434, 2447, 2448, 2449, 2456, 2462, 2463, 2467, 2468, 2471, 2477, 2478, 2482, 2486, 2497, 2502, 2514, 2532, 2541, 2559, 2581, 2585, 2590, 2602, 2605, 2609, 2614, 2624, 2637, 2652, 2660, 2661, 2669, 2711, 2724, 2729, 2733, 2735, 2739, 2747, 2756, 2761, 2779, 2783, 2788, 2804, 2811, 2837, 2841, 2870, 2871, 2876, 2882, 2886, 2888, 2889, 2917, 29

In [6]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.create -t "data/train_exp_exo.jsonl" -m "curie:ft-personal-2023-03-30-01-02-54"

Upload progress: 100%|█████████████████████| 1.15M/1.15M [00:00<00:00, 898Mit/s]
Uploaded file from data/train_exp_exo.jsonl: file-Q1tU9AU5YGyyP60FSWqt16SG
Created fine-tune: ft-PWzGlLWcHCAUGDav52amvw7A
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-03-31 16:27:15] Created fine-tune: ft-PWzGlLWcHCAUGDav52amvw7A

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-PWzGlLWcHCAUGDav52amvw7A



In [8]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.follow -i ft-PWzGlLWcHCAUGDav52amvw7A

[2023-03-31 16:27:15] Created fine-tune: ft-PWzGlLWcHCAUGDav52amvw7A
[2023-03-31 16:29:01] Fine-tune costs $4.59
[2023-03-31 16:29:02] Fine-tune enqueued. Queue number: 0
[2023-03-31 16:29:03] Fine-tune started
[2023-03-31 16:36:53] Completed epoch 1/4
[2023-03-31 16:50:27] Completed epoch 3/4
[2023-03-31 16:57:31] Uploaded model: curie:ft-personal-2023-03-31-23-57-31
[2023-03-31 16:57:32] Uploaded result file: file-rtBTRP8Q6dB5w7N5naQlPmPi
[2023-03-31 16:57:32] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m curie:ft-personal-2023-03-31-23-57-31 -p <YOUR_PROMPT>


In [12]:
import openai
from lib.cipher import substitution_cipher
from nltk.translate.bleu_score import sentence_bleu

openai.api_key = os.getenv("OPENAI_API_KEY")

def translate(text, debug=False, temperature=0.5):
    """
    Translate text from English to Birrig
    
    Parameters
    ----------
    text: str|array
        The text to translate. If an array is passed, each element will be translated.

    Returns
    ------- 
    array
        An array of translations
        
    """
    if isinstance(text, list):
        text = [row + "\n\n###\n\n" for row in text]
        if len(text) > 20:
            # break it into chunks of 20 and call translate on each chunk
            # then combine the results
            result = []
            for i in range(0, len(text), 20):
                result += translate(text[i:i+20])
            return result

    else:
        text += "\n\n###\n\n"

    response = openai.Completion.create(
        model="curie:ft-personal-2023-03-31-23-57-31",
        prompt=text,
        temperature=temperature,  # 0.1 is very little randomness/creativity, 2 is very
        max_tokens=100,
        n=1,
        logprobs=debug and 5 or 0,
        frequency_penalty=0,
        presence_penalty=0,
        stop=["\n###"]
    )
    result = []
    for choice in response.get('choices',[{}]):
        decoded = substitution_cipher(choice.get('text').strip(), encode=False).strip()
        result.append(decoded)
        if debug:
            # convert logprobs to probabilities
            logprobs = choice.get('logprobs', {}).get('token_logprobs', [])
            tokens = [substitution_cipher(x, encode=False) for x in choice.get('logprobs', {}).get('tokens', [])]
            probs = [10**logprob for logprob in logprobs]
            # merge the probabilities with the tokens
            probs = list(zip(tokens, probs))

            print("PROBS: ", probs)
            print("TRANSLATION: ", choice.get('text').strip())
            print("DECODED: ", decoded)
            # result.append({
        #     'translation': choice.get('text').strip(),
        #     'decoded_translation': decoded,
        #     'bleu_score': sentence_bleu([text.split()], decoded.split())
        # })
    
    return result

In [17]:
translate("he ate locusts and honey", debug=True, temperature=0.5)

PROBS:  [(' An', 0.19381704148122922), ('d', 0.9999375251343363), (' h', 0.9816975383876909), ('e', 0.8937931213540017), (' t', 0.39436651262285255), ('o', 0.9995017017871544), ('ok', 0.999938618695518), (' a', 0.19677079034393066), (' m', 0.9233867533675446), ('ea', 0.5842818692471377), ('l', 0.9949104525580819), (' of', 0.7289083746522869), (' l', 0.5435563453989167), ('oc', 0.9602171136562743), ('us', 0.9870646120746793), ('h', 0.39752965336046464), ('er', 0.34751015488980536), ('s', 0.11038999152475701), (' an', 0.9985136895654712), ('d', 0.999999566151524), (' h', 0.9970779777693828), ('on', 0.3749223236651674), ('ey', 0.9801926553707301), ('.', 0.38907907973999173)]
TRANSLATION:  Erw xi luuc e niep us pukazxivz erw xurij.
DECODED:  And he took a meal of locushers and honey.


['And he took a meal of locushers and honey.']

In [20]:
translate(bible[bible['0']=='MRK 1:6'][['eng-web']].values[0][0], debug=False, temperature=0.001)

['And Joshua was clothed with a haired hair and a linen band who were down on his side; and he took a meal-hole and unle disease.']

In [21]:
# Get the column eng-web from the first 3 rows of validate then send it to translate
validate['translation'] = translate(validate['eng-web'].to_list())
validate

/var/folders/tq/94d0g6xx05x_h_z4rfdfh6bh0000gn/T/ipykernel_69408/591098392.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validate['translation'] = translate(validate['eng-web'].to_list())


,0,book,chapter,verse,eng-web,eng-asv,eng-kjv2006,engBBE,engWycliffe,birrig,translation
24284,MRK 1:1,MRK,1,1,The beginning of the Good News of Jesus Christ...,"The beginning of the gospel of Jesus Christ, t...","The beginning of the gospel of Jesus Christ, t...",The first words of the good news of Jesus Chri...,"The bigynnyng of the gospel of Jhesu Crist, th...",Lxi sovzl huvwz us lxi guuw rihz us Yizaz Kxvo...,The birth of the Good Nows of Jesus Chisoh.
24285,MRK 1:2,MRK,1,2,"As it is written in the prophets, “Behold, I s...","Even as it is written in Isaiah the prophet, B...","As it is written in the prophets, Behold, I se...",Even as it is said in the book of Isaiah the p...,"As it is writun in Ysaie, the prophete, Lo! Y ...",Imir ez ol oz zeow or lxi fuuc us Ozeoex lxi q...,"As it is said in the prophets, See, I send my ..."
24286,MRK 1:3,MRK,1,3,"the voice of one crying in the wilderness, ‘Ma...","The voice of one crying in the wilderness, Mak...","The voice of one crying in the wilderness, Pre...","The voice of one crying in the waste land, Mak...","The vois of a crier in desert, Make ye redi th...","Lxi muoki us uri kvjorg or lxi hezli perw, Nec...","The voice of one crying in the waste land, Mak..."
24287,MRK 1:4,MRK,1,4,John came baptizing in the wilderness and prea...,"John came, who baptized in the wilderness and ...","John did baptize in the wilderness, and preach...","John came, and gave baptism in the waste land,...","Joon was in desert baptisynge, and prechynge t...","Yuxr keni, erw gemi feqlozn or lxi hezli perw,...",And Joshua came back with the blessing of all ...
24288,MRK 1:5,MRK,1,5,All the country of Judea and all those of Jeru...,And there went out unto him all the country of...,And there went out unto him all the land of Ju...,And there went out to him all the people of Ju...,"And al the cuntre of Judee wente out to hym, a...",Erw lxivi hirl ual lu xon epp lxi qiuqpi us Ya...,"All Judaal and Josphegal went out to him, and ..."
...,...,...,...,...,...,...,...,...,...,...,...
24957,MRK 16:16,MRK,16,16,He who believes and is baptized will be saved;...,He that believeth and is baptized shall be sav...,He that believeth and is baptized shall be sav...,He who has faith and is given baptism will get...,"Who that bileueth, and is baptisid, schal be s...",Xi hxu xez seolx erw oz gomir feqlozn hopp gil...,Anyima who has faith in the blessing of the qn...
24958,MRK 16:17,MRK,16,17,These signs will accompany those who believe: ...,And these signs shall accompany them that beli...,And these signs shall follow them that believe...,And these signs will be with those who have fa...,"And these tokenes schulen sue hem, that bileue...",Erw lxizi zogrz hopp fi holx lxuzi hxu xemi se...,These signs will be with them; and they will s...
24959,MRK 16:18,MRK,16,18,they will take up serpents; and if they drink ...,"they shall take up serpents, and if they drink...",They shall take up serpents; and if they drink...,"They will take up snakes, and if there is pois...",thei schulen do awei serpentis; and if thei dr...,"Lxij hopp leci aq zreciz, erw os lxivi oz quoz...","They will take up stones, and if they take any..."
24960,MRK 16:19,MRK,16,19,"So then the Lord, after he had spoken to them,...","So then the Lord Jesus, after he had spoken un...","¶ So then after the Lord had spoken unto them,...","So then the Lord Jesus, after he had said thes...","And the Lord Jhesu, aftir he hadde spokun to h...","Zu lxir lxi Puvw Yizaz, esliv xi xew zeow lxiz...","And when the Lord had said this to them, he we..."


In [22]:
# drop rows that have a null value in any column
validate = validate.dropna()
validate['bleu_score'] = validate.apply(lambda row: sentence_bleu([row['eng-web'].split(), row['eng-asv'].split(), row['eng-kjv2006'].split(),row['engBBE'].split()], row['translation'].split()), axis=1)

/Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

In [23]:
# describe the bleu score column
validate['bleu_score'].describe()

count     6.730000e+02
mean      2.425462e-01
std       2.091894e-01
min      5.244712e-232
25%       4.470823e-78
50%       2.262854e-01
75%       3.830598e-01
max       1.000000e+00
Name: bleu_score, dtype: float64

## Analysis
This theory proposes that to avoid word for word translation start with the web english version and target the BBE english version.  That way it has to use alternate words to find the way.

It generally does okay and makes mostly readable text.  Some weaknesses observed are
 - might get distracted and start repeating itself
 - likes to start most verses with "And"

| Stat | Number
| ----- | ----- |
| Average bleu score | 0.386 |
| 75 percentile | 0.532 |

In [112]:
# foreach row in validate, print the column "0" and translation
for index, row in validate.iterrows():
    print(row['0'], row['translation'])

MRK 1:1 The first of the good news of Jesus Christ, the Son of God.
MRK 1:2 As it says in the prophets, See, I send my servant before your face, who will make ready your way before you;
MRK 1:3 The voice of one crying in the waste land, Make ready the way of the Lord, make his roads straight.
MRK 1:4 And John came giving baptism in the waste land, crying the baptism of punishment as most against sorts of sin.
MRK 1:5 And all the country of Judaea and all those of Jerusalem went out to him; and they were given baptism by him in the jourts of the Jordan, saying openly that they had done wrong.
MRK 1:6 And John was clothed in camel's hair and with a linen broken in his side: and he took food of evil spirits and holy hole.
MRK 1:7 His words were true, saying, And after me comes he who is greater than I, the shoes of whom I am not good enough to take down and put it out.
MRK 1:8 The baptism of water is crushing you in away: but he will put you baptism in the Holy Spirit.
MRK 1:9 On those da

In [72]:
translate("For God so loved the world that he gave his only Son, so that everyone who believes in him may not die but have eternal life.")

['For God had such love for the world that he gave his only son, so that a man who has faith in him may not come to death but have eternal life.']

In [76]:
translate("he said let there be light and there was light")



['He said, Let the light be then, and the light came:']

In [77]:
# Try some unknown words
translate("Bongo bongo I love you, gone to Venus with a hole in my shoe")

['Go on bed, I have love for you, and underto the seed of Ramon to have a hand in my shoes:']

In [78]:
translate ("Ship Pit! Pirate ahoy-lay!")

['The strength! The servants!']

In [79]:
translate("Sheep went baa")

['And keeping away,']

In [80]:
translate("Boat a brother on a mountain")

['To be hand on a brother on a mountain;']

In [84]:
translate("Be kind and play with rocks!")

['Be certain, do their stones and take their plaeses streed.']

In [ ]:
validation

In [85]:
translate("SOS! Ship overboard!! Lost ninty percent of people!")

KeyboardInterrupt: 